In [8]:
import pandas as pd
import numpy as np

In [20]:
data=pd.read_csv("data_cleaned.csv")

In [36]:
data.head()

,sentence,b_per,label,key_words,score
0,"Aaargh), but the fact that the raw material is...",Aaargh,0,aaargh fact raw material john grisham tale exc...,0.023077
1,I have my doubts after watching Aag.,Aag,0,doubts watching aag,0.000000
2,Enough of stupid characters in your movie like...,Aag,0,stupid characters movie like shiva aag,-0.800000
3,I cannot say that Aag is the worst Bollywood f...,Aag,0,cannot aag worst bollywood film havent seen bo...,0.027778
4,"overall, RGV's Aag is worth watching for Amita...",Aag,0,overall rgvs aag worth watching amitabhs solid...,0.231429


In [19]:
data.to_csv("data_cleaned.csv",index=False)

## clean_sentence

In [22]:
import re

def clean_text(text):

    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    text = text.lower()  # Convert text to lowercase
    return text

In [23]:
data['sentence_normalized'] = data['sentence'].apply(lambda x: clean_text(x))

## tokenizer

In [24]:
tokenizer = []  
for sentence in data["sentence_normalized"].astype(str):  # Ensure all reviews are strings
    # Tokenize the review: remove non-alphanumeric characters and split into words
    word_tokens = sentence.split()
    tokenizer.append(word_tokens)

In [25]:
data["tokenizer"]=tokenizer

In [26]:
from spacy.lang.en.stop_words import STOP_WORDS

In [27]:
deselect_stop_words = ['n\'', 'love', 'hate', 'like', 'dislike', 'not', '’t', 'never', 'nobody', 'n’t','cannot','nothing','no']
stop_words = set(STOP_WORDS)
for w in deselect_stop_words:
    if w in stop_words:
        stop_words.remove(w)
    else:
         continue

# Now, stop_words contains the default stopwords excluding the ones we deselected
print(stop_words)

{'himself', 'after', 'might', 'whenever', 'twenty', 'still', 'all', 'seems', "'re", 'each', 'either', 'hundred', 'put', 'amongst', 'nevertheless', 'last', 'eight', 'forty', 'while', 'otherwise', 'now', 'first', 'take', 'than', 'may', 'anyone', 'ourselves', 'becoming', 'next', 'one', 'who', 'yet', 'becomes', '’ve', 'call', 'she', 'front', 'you', 'through', 'out', 'by', 'fifty', '‘ll', '‘d', 'whose', 'least', 'become', 'six', 'some', 'once', 'does', 'someone', 'together', 'except', 'thence', '‘ve', 'your', 'since', 'hereupon', 'hereby', 'enough', 'before', 'beforehand', 'however', 'every', 'thus', 'therein', 'several', 'around', 'many', 'of', 'have', '‘re', '’ll', 'herself', 'using', 'whither', 'above', 'itself', 'yours', 'being', 'noone', 'off', "'d", 'are', 'alone', 'whom', 'although', 'hereafter', 'could', 'or', 'them', 'various', 'be', 'should', 'both', 'their', 'would', 'something', 'against', 'somehow', 'beyond', 'sometime', 'between', 'top', '’re', 'nine', 'along', 'anywhere', 'wi

In [28]:
key_words = []
for comment in data["tokenizer"]:
    filteredComment = [w for w in comment if not (w in stop_words or len(w) <= 1)]  # Remove stop words and short words (<=2)
    key_words.append(' '.join(filteredComment))

In [29]:
data["key_words"]=key_words

In [30]:
from textblob import TextBlob

In [31]:
score = []
for i in data["key_words"]:
    score.append(TextBlob(i).sentiment.polarity)

In [32]:
data["score"]=score

In [37]:
data.to_csv("data_score.csv")

In [37]:

def get_central_sentence(group):
    # Find the median score for the group
    median_score = group['score'].median()
    # Calculate the absolute difference between the score and the median score
    group['abs_diff'] = (group['score'] - median_score).abs()
    # Find the row with the smallest absolute difference
    central_sentence = group.loc[group['abs_diff'].idxmin()]
    return central_sentence[['b_per', 'sentence', 'score']]

# Apply the function to each group
result = data.groupby('b_per').apply(get_central_sentence).reset_index(drop=True)

# Display the result
print(result)


         b_per                                           sentence     score
0       Aaargh  Aaargh), but the fact that the raw material is...  0.023077
1          Aag  I cannot say that Aag is the worst Bollywood f...  0.027778
2         Aage  and Robert Lloyd and Aage Haugland both play a... -0.050000
3       Aakash  I watched this movie on march 21 this year.Mus...  0.156731
4      Aakrosh  After 'Aakrosh' , this was the second film for...  0.000000
...        ...                                                ...       ...
16346     yeon  I know the actress Do-yeon Jeon got the Cannes...  1.000000
16347  yiannis  Perhaps, one of the most important and enjoyab...  0.313393
16348      you  If you are interested in making a reality film...  0.225000
16349     yway  ANyway, eventually Batman discovers that he's ...  0.000000
16350       zo  But when he himself is hustled by a scraggy, w...  0.084375

[16351 rows x 3 columns]


In [38]:
result.head()

,b_per,sentence,score
0,Aaargh,"Aaargh), but the fact that the raw material is...",0.023077
1,Aag,I cannot say that Aag is the worst Bollywood f...,0.027778
2,Aage,and Robert Lloyd and Aage Haugland both play a...,-0.050000
3,Aakash,I watched this movie on march 21 this year.Mus...,0.156731
4,Aakrosh,"After 'Aakrosh' , this was the second film for...",0.000000


In [39]:
result.to_csv("Final.csv")

## Evaluation 

In [44]:
data=pd.read_csv("data_score.csv")
data.head()

,Unnamed: 0,sentence,b_per,label,key_words,score
0,0,"Aaargh), but the fact that the raw material is...",Aaargh,0,aaargh fact raw material john grisham tale exc...,0.023077
1,1,I have my doubts after watching Aag.,Aag,0,doubts watching aag,0.000000
2,2,Enough of stupid characters in your movie like...,Aag,0,stupid characters movie like shiva aag,-0.800000
3,3,I cannot say that Aag is the worst Bollywood f...,Aag,0,cannot aag worst bollywood film havent seen bo...,0.027778
4,4,"overall, RGV's Aag is worth watching for Amita...",Aag,0,overall rgvs aag worth watching amitabhs solid...,0.231429


In [40]:
data.describe()

,label,textblob_score
count,149652.000000,149652.000000
mean,0.603306,0.110864
std,0.489213,0.307088
min,0.000000,-1.000000
25%,0.000000,0.000000
50%,1.000000,0.050000
75%,1.000000,0.282500
max,1.000000,1.000000


In [35]:
data.drop(columns=['i_per', 'sentence_normalized', 'tokenizer'], inplace=True)

In [45]:
data.rename(columns={'score': 'textblob_score'}, inplace=True)

## model_1 textblob

In [46]:
sentiment_mapping = []
for i in data["key_words"]:
    vs = TextBlob(i).sentiment.polarity  # Get the polarity score
    if vs > 0:
        sentiment_mapping.append((1,vs))
    elif vs < 0:
        sentiment_mapping.append((0,vs))
    else:
        sentiment_mapping.append((0,vs))

In [47]:
data["textblob_label"]=[sentiment[0] for sentiment in sentiment_mapping]
data["textblob_score"]=[sentiment[1] for sentiment in sentiment_mapping]
data.head(30) 

,Unnamed: 0,sentence,b_per,label,key_words,textblob_score,textblob_label
0,0,"Aaargh), but the fact that the raw material is...",Aaargh,0,aaargh fact raw material john grisham tale exc...,0.023077,1
1,1,I have my doubts after watching Aag.,Aag,0,doubts watching aag,0.000000,0
2,2,Enough of stupid characters in your movie like...,Aag,0,stupid characters movie like shiva aag,-0.800000,0
3,3,I cannot say that Aag is the worst Bollywood f...,Aag,0,cannot aag worst bollywood film havent seen bo...,0.027778,1
4,4,"overall, RGV's Aag is worth watching for Amita...",Aag,0,overall rgvs aag worth watching amitabhs solid...,0.231429,1
5,5,And if you valued music more than story the li...,Aag,1,valued music story list simply grow beautiful ...,0.416667,1
6,6,What Aag does is take these characters and mes...,Aag,0,aag characters mess badly ull need contest pic...,-0.625000,0
7,7,"Parsifal (1982) Starring Michael Kutter, Armin...",Aage,1,parsifal starring michael kutter armin jordan ...,0.352381,1
8,8,and Robert Lloyd and Aage Haugland both play a...,Aage,1,robert lloyd aage haugland play sing gurnemanz...,-0.050000,0
9,9,I watched this movie on march 21 this year.Mus...,Aakash,0,watched movie march yearmust disappointmentbut...,0.156731,1


In [51]:
accuracy_textblob = (data['label'] == data['textblob_label']).mean()
print(accuracy_textblob)

0.5880308983508407


## model_2 Gensim


## model_3 nltk